## Setup


[Setup Python Virtual Environment in VSCode](/venv_setup.md)


**Security Note:** Using fixed package versions ensures stability and reproducibility, but may expose you to security vulnerabilities if not regularly updated. Balance stability and security by periodically reviewing and updating packages.

In [ ]:
# Install required packages in a virtual environment (Currently only tested with Python 3.12.7)
%pip install --upgrade pip
%pip install pandas==2.2.2
%pip install requests==2.32.3

## Download JLCPCB Parts CSV File 

In [2]:
import requests
import os


def download_file(url, filename):
    try:
        # Check if the file already exists
        if os.path.exists(filename):
            # Delete the existing file
            os.remove(filename)
            print(f"Deleted existing file: {filename}")

        response = requests.get(f"{url}/{filename}", stream=True)
        response.raise_for_status()  # Raise an exception for bad status codes
        with open(filename, "wb") as f:
            for chunk in response.iter_content(None):
                f.write(chunk)
        print(f"Downloaded {url}/{filename} to {filename}")
    except requests.RequestException as e:
        print(f"Download {url} failed: {e}")


# URL
url = "https://cdfer.github.io/jlcpcb-parts-database"

# Download the file
download_file(url, "jlcpcb-components-basic-preferred.csv")

Deleted existing file: jlcpcb-components-basic-preferred.csv
Downloaded https://cdfer.github.io/jlcpcb-parts-database/jlcpcb-components-basic-preferred.csv to jlcpcb-components-basic-preferred.csv


## Lowest Cost Per Pin Basic Components

Calculates the cost per joint for electronic components from a JLCPCB CSV file.

Filters components with at least 8 pins, calculates the cost per joint based on price and joint type (SMT or hand-soldered), and prints the top 100 components with the lowest cost per joint.

FYI JLCPCB calls a pin a joint (e.g. a solder joint)


In [ ]:
import pandas as pd
import json

# Constants
MINIMUM_PINS = 8
SMT_JOINT_COST = 0.0017
HAND_SOLDER_JOINT_COST = 0.0173

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("jlcpcb-components-basic-preferred.csv")


# Define a function to calculate the cost per joint
def calculate_cost_per_joint(row):
    price = json.loads(row["price"])[0]["price"]
    joint_cost = HAND_SOLDER_JOINT_COST if "Plugin" in row["package"] else SMT_JOINT_COST
    return (float(price) + (row["joints"] * joint_cost)) / row["joints"]


# Calculate cost per joint for each component
df["cost_per_joint"] = df.apply(calculate_cost_per_joint, axis=1)

# Filter parts with more than MINIMUM_PINS joints
df_filtered = df[df["joints"] >= MINIMUM_PINS]

# Sort the components by cost per joint
df_sorted = df_filtered.sort_values(by="cost_per_joint")

# Print the top 100 components with the lowest cost per joint
for index, row in df_sorted.head(100).iterrows():
    print(
        f"USD/Pin = {row['cost_per_joint']:.3f} "
        f"https://jlcpcb.com/partdetail/C{row['lcsc']} "
        f"Pins: {row['joints']} {row['mfr']} ({row['description']})"
    )

USD/Pin = 0.002 https://jlcpcb.com/partdetail/C25725 Pins: 8 4D02WGJ0103TCE (4 ±5% 10kΩ 62.5mW ±200ppm/℃ 0402x4  Resistor Networks, Arrays ROHS)
USD/Pin = 0.002 https://jlcpcb.com/partdetail/C20197 Pins: 8 4D03WGJ0102T5E (4 ±5% 1kΩ 62.5mW ±200ppm/℃ 0603x4  Resistor Networks, Arrays ROHS)
USD/Pin = 0.002 https://jlcpcb.com/partdetail/C1980 Pins: 8 4D03WGJ0472T5E (4 ±5% 4.7kΩ 62.5mW ±200ppm/℃ 0603x4  Resistor Networks, Arrays ROHS)
USD/Pin = 0.002 https://jlcpcb.com/partdetail/C2991336 Pins: 8 RTA02-4D512JTH (4 ±5% 5.1kΩ 62.5mW ±200ppm/℃ 0402x4  Resistor Networks, Arrays ROHS)
USD/Pin = 0.002 https://jlcpcb.com/partdetail/C25510 Pins: 8 4D03WGJ0471T5E (4 ±5% 470Ω 62.5mW ±200ppm/℃ 0603x4  Resistor Networks, Arrays ROHS)
USD/Pin = 0.002 https://jlcpcb.com/partdetail/C29718 Pins: 8 4D03WGJ0103T5E (4 ±5% 10kΩ 62.5mW ±200ppm/℃ 0603x4  Resistor Networks, Arrays ROHS)
USD/Pin = 0.004 https://jlcpcb.com/partdetail/C7420374 Pins: 10 H5VU25U (4A 14V 56W 6V Unidirectional 5V DFN2510-10L  ESD and Su